In [1]:
import pandas as pd

In [33]:
df = pd.read_csv('transactions_dataset.csv', sep=';')

In [4]:
df.shape

(63319315, 8)

In [5]:
df.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051


In [7]:
df.columns

Index(['date_order', 'date_invoice', 'product_id', 'client_id', 'sales_net',
       'quantity', 'order_channel', 'branch_id'],
      dtype='object')

In [51]:
count_rows = df.groupby(['date_order', 'client_id', 'product_id']).size()
print(count_rows)


date_order  client_id  product_id
2017-09-22  403403     56891         1
                       532445        1
                       2995430       1
            585799     183015        1
                       452441        1
                                    ..
2019-09-22  2265063    1912743       1
                       2146061       1
                       2772261       1
                       2863707       1
                       2885889       1
Length: 54760558, dtype: int64


check returns - not grouped by date


In [11]:
# Convert 'date_order' to datetime if it's not already in datetime format
df['date_order'] = pd.to_datetime(df['date_order'])

In [ ]:
# Create a column 'is_returned' indicating if a product for a client was returned later
# df['is_returned'] = df.groupby(['client_id', 'product_id'])['sales_net'].transform(lambda x: (x < 0).any()).astype(int)
# df

In [12]:
# Count the number of returns for each client-product pair
return_counts = df[df['sales_net'] < 0].groupby(['client_id', 'product_id']).size().rename('return_count')

In [13]:
# Calculate total purchases made by each client
total_purchases = df.groupby(['client_id']).size().rename('total_purchases')

In [14]:
# Merge return_counts and total_purchases back to the main DataFrame based on 'client_id' and 'product_id'
df = df.merge(return_counts, on=['client_id', 'product_id'], how='left')
df = df.merge(total_purchases, on=['client_id', 'product_id'], how='left')

In [15]:
# Fill NaN values in 'return_count' and 'total_purchases' columns with 0 (for pairs with no returns)
df['return_count'] = df['return_count'].fillna(0)
df['total_purchases'] = df['total_purchases'].fillna(0)
df

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,0.0,1
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,0.0,4
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,0.0,2
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,0.0,2
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,0.0,1
...,...,...,...,...,...,...,...,...,...,...
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.91300,5,by phone,7682,1.0,1
63319311,2018-12-14,2020-07-15,908253,880496,27.62760,1001,by phone,9375,1.0,2
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.60984,101,by phone,3014,1.0,2
63319313,2018-05-23,2020-11-27,2319965,1287654,70.49730,17,by phone,1015,1.0,2


In [21]:
df['ret_pct'] = df['return_count'] / df['total_purchases'].fillna(1)
df

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases,ret_pct
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,0.0,1,0.0
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,0.0,4,0.0
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,0.0,2,0.0
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,0.0,2,0.0
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.91300,5,by phone,7682,1.0,1,1.0
63319311,2018-12-14,2020-07-15,908253,880496,27.62760,1001,by phone,9375,1.0,2,0.5
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.60984,101,by phone,3014,1.0,2,0.5
63319313,2018-05-23,2020-11-27,2319965,1287654,70.49730,17,by phone,1015,1.0,2,0.5


In [25]:
df['ret_pct'].value_counts()

ret_pct
0.000000    50713374
0.500000     1761460
0.333333     1354947
0.250000      752444
0.200000      534505
              ...   
0.923077          13
0.909091          11
0.900000          10
0.888889           9
0.777778           9
Name: count, Length: 2884, dtype: int64

In [19]:
df[(df['client_id'] == 835089) & (df['product_id'] == 2520527)]


,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,0.0,4
7414736,2017-12-20,2017-12-20,2520527,835089,0.00000,31,at the store,10279,0.0,4
7800173,2017-12-20,2017-12-20,2520527,835089,169.29472,31,at the store,10279,0.0,4
8054358,2017-12-27,2017-12-27,2520527,835089,16.39440,3,at the store,10279,0.0,4


ordered date returns

In [ ]:
# Convert 'date_order' to datetime if it's not already in datetime format
df['date_order'] = pd.to_datetime(df['date_order'])

# Create a column 'is_returned' indicating if a product for a client was returned later for each day and each client-product pair
df['is_returned'] = df.groupby([df['date_order'].dt.date, 'client_id', 'product_id'])['sales_net'].transform(lambda x: (x < 0).any()).astype(int)

# Count the number of returns for each day, client, and product
return_counts = df[df['sales_net'] < 0].groupby([df['date_order'].dt.date, 'client_id', 'product_id']).size().rename('return_count')

# Count the total number of purchases for each day and each client
total_purchases = df.groupby([df['date_order'].dt.date, 'client_id']).size().rename('total_purchases')

# Merge return_counts and total_purchases back to the main DataFrame based on 'date_order', 'client_id', and 'product_id'
df = df.merge(return_counts, on=['date_order', 'client_id', 'product_id'], how='left')
df = df.merge(total_purchases, on=['date_order', 'client_id'], how='left')

# Calculate return percentage (ret_pct) for each day and each client-product pair
df['ret_pct'] = df['return_count'] / df['total_purchases'].fillna(1)  # Use fillna(1) to handle divisions by zero

# Fill NaN values in 'return_count' and 'total_purchases' columns with 0 (for pairs with no returns)
df['return_count'] = df['return_count'].fillna(0)
df['total_purchases'] = df['total_purchases'].fillna(0)

print(df)


In [34]:
# Count the number of returns for each day, client, and product
return_counts = df[df['sales_net'] < 0].groupby([df['date_order'], 'client_id', 'product_id']).size().rename('return_count')

In [35]:
# Count the total number of purchases for each day and each client
total_purchases = df.groupby([df['date_order'], 'client_id']).size().rename('total_purchases')

In [36]:
# Merge return_counts and total_purchases back to the main DataFrame based on 'date_order', 'client_id', and 'product_id'
df = df.merge(return_counts, on=['date_order', 'client_id', 'product_id'], how='left')
df = df.merge(total_purchases, on=['date_order', 'client_id'], how='left')

In [37]:
df['return_count'] = df['return_count'].fillna(0)
df['total_purchases'] = df['total_purchases'].fillna(0)
df

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,0.0,18
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,0.0,5
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,0.0,5
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,0.0,3
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,0.0,2
...,...,...,...,...,...,...,...,...,...,...
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.91300,5,by phone,7682,1.0,43
63319311,2018-12-14,2020-07-15,908253,880496,27.62760,1001,by phone,9375,1.0,24
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.60984,101,by phone,3014,1.0,19
63319313,2018-05-23,2020-11-27,2319965,1287654,70.49730,17,by phone,1015,1.0,8


In [40]:
df['ret_pct'] = df['return_count'] / df['total_purchases'].fillna(0)

In [41]:
df

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases,ret_pct
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,0.0,18,0.000000
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,0.0,5,0.000000
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,0.0,5,0.000000
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,0.0,3,0.000000
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,0.0,2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.91300,5,by phone,7682,1.0,43,0.023256
63319311,2018-12-14,2020-07-15,908253,880496,27.62760,1001,by phone,9375,1.0,24,0.041667
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.60984,101,by phone,3014,1.0,19,0.052632
63319313,2018-05-23,2020-11-27,2319965,1287654,70.49730,17,by phone,1015,1.0,8,0.125000


In [48]:
df['ret_pct'].value_counts().sort_values(ascending=False)

ret_pct
0.000000    58173984
0.250000      253479
0.166667      206307
0.125000      190199
0.500000      182740
              ...   
0.002092           3
0.001757           2
0.001481           2
0.005195           2
0.002584           1
Name: count, Length: 1796, dtype: int64

In [50]:
df[(df['ret_pct']> 0) & (df['sales_net'] > 0)]

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases,ret_pct
23,2017-09-25,2017-09-25,123753,810674,101.714118,35,by phone,2732,1.0,46,0.021739
87,2017-09-29,2017-09-29,3000784,624671,2.290800,3,by phone,7251,1.0,10,0.100000
90,2017-09-27,2017-09-27,2179202,783783,12.711640,41,at the store,2472,1.0,68,0.014706
100,2017-09-25,2017-09-25,2302298,2103907,182.609267,13,by phone,688,1.0,37,0.027027
207,2017-09-26,2017-09-26,482165,1789729,0.966000,7,at the store,3947,1.0,50,0.020000
...,...,...,...,...,...,...,...,...,...,...,...
63319282,2018-06-28,2020-02-20,1184623,1606180,2760.000000,3,by phone,3607,1.0,10,0.100000
63319287,2019-08-13,2020-05-22,2904700,798548,1907.160000,3,by phone,6184,1.0,55,0.018182
63319299,2019-09-11,2020-05-07,929509,1511287,281.642667,7,by phone,2449,1.0,70,0.014286
63319311,2018-12-14,2020-07-15,908253,880496,27.627600,1001,by phone,9375,1.0,24,0.041667


In [43]:
df[df['sales_net']<0]

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,return_count,total_purchases,ret_pct
129,2017-09-26,2017-09-26,121848,1789606,-182.792500,25,at the store,5395,1.0,26,0.038462
219,2017-09-26,2017-09-26,1435071,1791722,-169.456272,51,at the store,5167,1.0,10,0.100000
621,2017-09-25,2017-09-27,693904,2167021,-36.901200,3,online,6184,1.0,48,0.020833
936,2017-09-25,2017-09-26,133038,480303,-64.446000,5,at the store,210,1.0,28,0.035714
1054,2017-09-25,2017-09-26,235896,1324117,-180.113000,5,by phone,10449,1.0,4,0.250000
...,...,...,...,...,...,...,...,...,...,...,...
63319308,2019-06-11,2020-06-09,293157,826473,-123.951600,3,at the store,2429,1.0,2,0.500000
63319309,2019-08-30,2020-06-11,1793357,2180864,-16.698000,3,by phone,8159,1.0,230,0.004348
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.913000,5,by phone,7682,1.0,43,0.023256
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.609840,101,by phone,3014,1.0,19,0.052632
